# 4 - Extracting MFCCs from My iTunes Library
## Training & Classification
Last post, I referenced a great [MIR resource](http://musicinformationretrieval.com/genre_recognition.html) that already had outlined a process to do what I'm currently trying to do.

To recap some of the methods I learned, each training song is broken up into frames and the MFCCs of each _**frame**_ became a training sample. Each song yielded around ~5000 training samples (5000 frames) and all of them were labelled with the song.

Test samples were a shorter excerpt from the song and consisted of ~430 frames. Each frame became a test sample and were classified individually. This means that _**a single song could have been classified as multiple classes**_. I could see a world where and R&B or Hip Hop song has a nice Jazz break in the middle, and if he just happened to take our test sample from that break, those frames could be classified as Jazz.

The consequences here are two-fold. Not only do we run the risk of misclassifying a song, we run the risk of actually feeding _**incorrect**_ training data to our model! If we happen to catch the Jazz break of that R&B song and we classify that as R&B, it will cloud our model a bit.

## Generating Training Data
Before I start my script, I just want to think a bit more about how much data I'm actually going to have. It's a bit crazy because I didn't realize every single frame would be a different training sample. The 1 minute librosa example clip yielded 2657 frames... That's about 44 frames per second sampled, which comes out to about 1 frame every ~20ms or so.

The average song in my library is about 5 minutes, and I have just shy of 4000 songs. We're talking

$$4000\ Songs\times5\ Minutes\times\ 60\ Seconds\times44\ Frames=52,800,000\ Samples$$

Lol ok, and each of those samples have however many MFCC coefficients I want to use... at 10 MFCCs we would literally have over 500M individual data points. If we used 8 bits to store each number (these MFCCs are floats so it's probably even more), we'd have 4GB of data. I'm not so worried about the storage space as so much the amount of data our model will have to handle. Generally, I've dealt with \<2GB of data in a data analysis type of setting because I usually don't have more than 8GB of memory to work with (and on the laptop I'm using now, I only have 4) so I'll have to truncate this data set a bit further.

This is, of course, just an estimate, but what would this number look like if I only took 30 second extracts of each song? If I think of an average song, how much of it is... you know... the same beat... same words... same instruments... not for all songs, but if we're looking at Pop and R&B we'd definitely see a lot of repetition. We'd probably only need like 15 seconds of a song to recognize what genre it is! Maybe not even that!

$$4000\ Songs\times15\ Seconds\times44\ Frames=2,464,000\ Samples$$

This, my computer can probably handle, but I'm a bit hesitant the models themselves can handle it... This will be a good opportunity to try out different models and see which ones really get bogged down by computation. Let's just go with 15 seconds for now.

Next issue... where do I take this 15 second sample? Probably not right at the beginning or end because the intros and outros of songs vary quite often with how the rest of the song (arguably the main parts of the song that identifies with its genre) may sound. There are generally breaks and bridges that also may take on a bit of a different tone (that is, for those genres which even have breaks and bridges). Because I listen to a ton of dance music, I know that generally the first 30 seconds are an intro, then the real feel of the song kicks in. For R&B and pop, there are many examples I can think of where the real beat starts right away and others where there is an intro. Hip Hop generally gets right to the point. Folk and Rock are quite variable and I probably don't listen to enough of those genres to really generalize the structure. Ambient is the same throughout. I feel like if I start about 45 seconds in, I'd be skipping the intro in most cases and we'd probably be somewhere in the first verse, bridge, or chorus for those songs where this type of structure applies. I'll just go with this for now and tune this later if need be.

With this, I should be able to start writing my script.

## Training & Test Split
I'll reserve 10% of my library for test data.